In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup 
from lxml import etree as ET
pd.options.display.max_columns=50

In [2]:
def household(household_df,persons,mzone):
    
    household_df=household_df.sort_values(by=['hhno'])
    household_df['hhno']=household_df['hhno'].astype(str)
    
    persons['hhno']=persons['hhno'].astype(str)
    persons['pno']=persons['pno'].astype(str)

    persons['person_id']=persons['hhno']+'-'+persons['pno']

    
    household_w_pop=pd.merge(household_df,persons[['hhno','person_id']],on=['hhno'],how='left')
    
    hh_pop_mzone=pd.merge(household_w_pop,mzone,left_on=['hhparcel'],right_on=['MAZID'],how='left')
    
    hh_pop_mzone=hh_pop_mzone[['hhno','person_id','hhvehs','hhincome','X_COORD','Y_COORD']]
    
    return hh_pop_mzone

In [3]:
def population(household_df,persons):
    
    household_df['hhno']=household_df['hhno'].astype(str)

    
    household_w_pop=pd.merge(household_df,persons[['hhno','person_id','pagey','pgend']],
                             on=['hhno'],how='right')
    
    household_w_pop=household_w_pop[['person_id','hhno','pagey','pgend','hhvehs']]
    
    return household_w_pop

In [4]:
households=pd.read_table('../SF-CHAMP Outputs/_household_2.dat',sep='\t',header='infer')
person=pd.read_table('../SF-CHAMP Outputs/_person_2.dat',sep='\t',header='infer')
mzone=pd.read_csv('../SF-CHAMP Outputs/mzone_short.csv')

In [5]:
len(person)

7332674

In [5]:
#trip=pd.read_table('../SF-CHAMP Outputs/_trip_2.dat',sep='\t',header='infer')

In [6]:
# trip['hhno']=trip['hhno'].astype(str)
# trip['pno']=trip['pno'].astype(str)

# trip['person_id']=trip['hhno']+'-'+trip['pno']

# trip_ei_person=trip[(trip['otaz']<1000)|(trip['dtaz']<1000)]
# trip_ei_person=trip_ei_person[['person_id','hhno']]

# trip_int_int_person=trip[(trip['otaz']<1000)&(trip['dtaz']<1000)]
# trip_int_int_person=trip_int_int_person[['person_id','hhno']]
# trip_int_int_person.head()

In [6]:
trip_ei_person=pd.read_csv('../Script/agent_id.csv')
trip_ei_person.head()

,Unnamed: 0,person_id
0,0,1-1
1,1,1-1
2,2,10-1
3,3,10-1
4,4,100-1


In [9]:
len(trip_ei_person.person_id.unique())

1140347

In [7]:
person['hhno']=person['hhno'].astype(str)
person['pno']=person['pno'].astype(str)

person['person_id']=person['hhno']+'-'+person['pno']

In [33]:
households.head()

,hhno,fraction_with_jobs_outside,hhsize,hhvehs,hhwkrs,hhftw,hhptw,hhret,hhoad,hhuni,hhhsc,hh515,hhcu5,hhincome,hownrent,hrestype,hhparcel,zone_id,hhtaz,hhexpfac,samptype
3,9,0.04796,1,1,1,1,0,0,0,0,0,0,0,30020,11117,3,720,0,1,1,11
4,5,0.04796,1,1,0,0,0,1,0,0,0,0,0,18373,11117,2,720,0,1,1,11
5,15,0.04796,2,1,0,0,0,2,0,0,0,0,0,9165,11488,3,20,0,1,1,11
6,20,0.04796,2,1,1,1,0,0,1,0,0,0,0,36487,11117,1,720,0,1,1,11
7,1,0.04796,1,0,0,0,0,0,1,0,0,0,0,942,11117,2,720,0,1,1,11


In [8]:
person_ei=person[person['person_id'].isin(trip_ei_person.person_id.unique())]

In [9]:
pop_df=population(households,person_ei)
pop_df.head()

,person_id,hhno,pagey,pgend,hhvehs
0,9-1,9,54,1,1
1,5-1,5,79,2,1
2,15-2,15,66,2,1
3,20-2,20,55,1,1
4,1-1,1,55,2,0


In [10]:
pop_df[pop_df['person_id']=='456251-3']

,person_id,hhno,pagey,pgend,hhvehs


In [10]:

pop_df['gender']=np.where(pop_df['pgend']==1,
                                'male',
                                'female')
pop_df['pagey']=pop_df['pagey'].astype(str)


In [11]:
pop_df['excluded_mode']=np.where(pop_df['hhvehs']==0,
                                'car',
                                '')
pop_df['excluded_mode'].describe()

count     1269986
unique          2
top              
freq      1091711
Name: excluded_mode, dtype: object

In [15]:
parser = ET.XMLParser()

template = ET.parse('Templates/persons_attribute.xml',parser)

root = template.getroot()

In [30]:
list(range(0,51,5))

[0, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50]

In [ ]:
new_df

In [16]:
pop_df[pop_df['person_id']=='9-1']['pagey'][0]

'54'

In [17]:
len(pop_df)

1140347

In [123]:
pop_df.iloc[815000:825015]

,person_id,hhno,pagey,pgend,hhvehs,excluded_mode,gender
815000,2815547-1,2815547,18,1,0,car,male
815001,2836370-1,2836370,76,1,0,car,male
815002,2806576-1,2806576,47,2,1,,female
815003,2836658-1,2836658,19,2,1,,female
815004,2807680-1,2807680,18,2,0,car,female
815005,2822555-1,2822555,82,1,1,,male
815006,2817994-1,2817994,65,2,0,car,female
815007,2814016-1,2814016,33,1,1,,male
815008,2834891-1,2834891,46,1,1,,male
815009,2837003-1,2837003,54,1,1,,male


In [98]:
pop_df[pop_df['person_id']=='9-1'].values[0,2]

'54'

In [12]:
def create_xml_pop(df_all,size):

    for i in range(0,len(df_all)+1,size):
    
        j=i+size

        test_df=df_all.iloc[i:j].reset_index()

        parser = ET.XMLParser(remove_blank_text=True)

        template = ET.parse('../SF_CHAMP_Converted/persons_attribute.xml',parser)

        root = template.getroot()
        
        for person in test_df.person_id.unique():
            
            new_df=test_df[test_df['person_id']==person]
            
            person_tag = ET.SubElement(root, 'object') 
            person_tag.set('id',person)


            age_tag = ET.SubElement(person_tag, 'attribute')
            age_tag.set('name','age')
            age_tag.set('class','java.lang.Integer')
            age_tag.text=new_df.values[0,new_df.columns.get_loc('pagey')]

            sex_tag = ET.SubElement(person_tag, 'attribute')
            sex_tag.set('name','sex')
            sex_tag.set('class','java.lang.String')
            sex_tag.text=new_df.values[0,new_df.columns.get_loc('gender')]

            modes_tag = ET.SubElement(person_tag, 'attribute')
            modes_tag.set('name','excluded-modes')
            modes_tag.set('class','java.lang.String')
            modes_tag.text=new_df.values[0,new_df.columns.get_loc('excluded_mode')]

            root.append(person_tag)
        
#         for row in range(0,size):
            

#             person_tag = ET.SubElement(root, 'object') 
#             person_tag.set('id',test_df.values[row,test_df.columns.get_loc('person_id')])


#             age_tag = ET.SubElement(person_tag, 'attribute')
#             age_tag.set('name','age')
#             age_tag.set('class','java.lang.Integer')
#             age_tag.text=test_df.values[row,test_df.columns.get_loc('pagey')]

#             sex_tag = ET.SubElement(person_tag, 'attribute')
#             sex_tag.set('name','sex')
#             sex_tag.set('class','java.lang.String')
#             sex_tag.text=test_df.values[row,test_df.columns.get_loc('gender')]

#             modes_tag = ET.SubElement(person_tag, 'attribute')
#             modes_tag.set('name','excluded-modes')
#             modes_tag.set('class','java.lang.String')
#             modes_tag.text=test_df.values[row,test_df.columns.get_loc('excluded_mode')]

#             root.append(person_tag)


        tree=ET.ElementTree(root)
        tree.write('../SF_CHAMP_Converted/persons_attribute.xml', pretty_print=True, xml_declaration=True,   encoding="utf-8")

In [13]:
%time
create_xml_pop(pop_df,5000)

Wall time: 0 ns


In [14]:
households=households[households['hhno'].isin(pop_df.hhno.unique())]

In [15]:
df=household(households,person_ei,mzone)


<ipython-input-2-d76dddd0874f>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  persons['hhno']=persons['hhno'].astype(str)
<ipython-input-2-d76dddd0874f>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  persons['pno']=persons['pno'].astype(str)
<ipython-input-2-d76dddd0874f>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/

In [25]:
df.head()

,hhno,person_id,hhvehs,hhincome,X_COORD,Y_COORD
0,1,1-1,0,942,551662.441744,4.173739e+06
1,10,10-1,0,28888,551662.441744,4.173739e+06
2,100,100-1,3,48612,551662.441744,4.173739e+06
3,100,100-2,3,48612,551662.441744,4.173739e+06
4,100,100-3,3,48612,551662.441744,4.173739e+06


In [16]:
def create_xml(hh_pop_mzone,size):

    for i in range(0,len(hh_pop_mzone.hhno.unique())+1,size):
    
        j=i+size

        test_df=hh_pop_mzone[hh_pop_mzone['hhno'].isin(hh_pop_mzone.hhno.unique()[i:j])]

        parser = ET.XMLParser(remove_blank_text=True)

        template = ET.parse('../SF_CHAMP_Converted/household.xml',parser)

        root = template.getroot()

        for i in test_df['hhno'].unique():

            hh_id=str(i)

            household_tag = ET.SubElement(root, 'household') 
            household_tag.set('id',str(hh_id))


            members_tag = ET.SubElement(household_tag, 'members')

            df=test_df[test_df['hhno']==i]

            
            for person in df.person_id.unique():

                person_tag=ET.SubElement(members_tag,'personId')

                person_tag.set('refId',str(person))



            veh_list=[]
            if df.hhvehs.max()>0:
                x=df.hhvehs.max()
                vehicle_tag=ET.SubElement(household_tag, 'vehicles')

                for item in range(0,x):
                    vehicle_id='auto-'+hh_id+'-'+str(item)
                    veh_list.append(vehicle_id)

                for vehicles in veh_list:

                    vehicle_list_tag = ET.SubElement(vehicle_tag, 'vehicleDefinitionId')
                    vehicle_list_tag.set('refId',str(vehicles))

            income_tag=ET.SubElement(household_tag,'income')
            income_tag.set('period','year')
            income_tag.set('currency','usd')
            income_tag.text=str(df.hhincome.max())

            root.append(household_tag)


        tree=ET.ElementTree(root)
        tree.write('../SF_CHAMP_Converted/household.xml', pretty_print=True, xml_declaration=True,   encoding="utf-8")

In [15]:
df[df['hhno']=='456251']

,hhno,person_id,hhvehs,hhincome,X_COORD,Y_COORD
940890,456251,456251-1,4,122467,547275.302602,4.160566e+06


In [17]:
%%time
create_xml(df,5000)

Wall time: 51min 23s


In [18]:
df_hh=df.drop_duplicates(['hhno'])
df_hh.head()

,hhno,person_id,hhvehs,hhincome,X_COORD,Y_COORD
0,1,1-1,0,942,551662.441744,4.173739e+06
1,10,10-1,0,28888,551662.441744,4.173739e+06
2,100,100-1,3,48612,551662.441744,4.173739e+06
6,1000,1000-1,3,80081,547967.142735,4.173748e+06
11,10000,10000-1,2,56299,548116.858820,4.174127e+06


In [29]:
len(df_hh)

697579

In [19]:
def create_xml_attributes(hh_pop_mzone,size):

    for i in range(0,len(hh_pop_mzone.hhno.unique())+1,size):
    
        j=i+size

        test_df=hh_pop_mzone[hh_pop_mzone['hhno'].isin(hh_pop_mzone.hhno.unique()[i:j])]

        parser = ET.XMLParser(remove_blank_text=True)

        template = ET.parse('../SF_CHAMP_Converted/household_attributes.xml',parser)

        root = template.getroot()

        for i in test_df['hhno'].unique():

            hh_id=str(i)
    
            household_tag = ET.SubElement(root, 'object') 
            household_tag.set('id',str(hh_id))

            subset_df=test_df[test_df['hhno']==i]

            xcoord_tag = ET.SubElement(household_tag, 'attribute')
            xcoord_tag.set('name','homecoordx')
            xcoord_tag.set('class','java.lang.Double')
            xcoord_tag.text=str(subset_df.values[0,subset_df.columns.get_loc('X_COORD')])

            ycoord_tag = ET.SubElement(household_tag, 'attribute')
            ycoord_tag.set('name','homecoordy')
            ycoord_tag.set('class','java.lang.Double')
            ycoord_tag.text=str(subset_df.values[0,subset_df.columns.get_loc('Y_COORD')])

            housing_type_tag = ET.SubElement(household_tag, 'attribute')
            housing_type_tag.set('name','housingtype')
            housing_type_tag.set('class','java.lang.String')
            housing_type_tag.text='House'

            root.append(household_tag)


        tree=ET.ElementTree(root)
        tree.write('../SF_CHAMP_Converted/household_attributes.xml', pretty_print=True, xml_declaration=True,   encoding="utf-8")

In [20]:
%%time
create_xml_attributes(df_hh,5000)

Wall time: 44min 32s


In [164]:
%%time
parser = ET.XMLParser(remove_blank_text=True)

template = ET.parse('SF_CHAMP_Converted/household_int_int_attributes.xml',parser)

root = template.getroot()

for i in df['hhno'].unique():
    
    hh_id=str(i)
    
    household_tag = ET.SubElement(root, 'object') 
    household_tag.set('id',str(hh_id))

    subset_df=df[df['hhno']==i]
    
    xcoord_tag = ET.SubElement(household_tag, 'attribute')
    xcoord_tag.set('name','homecoordx')
    xcoord_tag.set('class','java.lang.Double')
    xcoord_tag.text=str(subset_df['X_COORD'].unique()[0])
    
    ycoord_tag = ET.SubElement(household_tag, 'attribute')
    ycoord_tag.set('name','homecoordy')
    ycoord_tag.set('class','java.lang.Double')
    ycoord_tag.text=str(subset_df['Y_COORD'].unique()[0])
    
    housing_type_tag = ET.SubElement(household_tag, 'attribute')
    housing_type_tag.set('name','housingtype')
    housing_type_tag.set('class','java.lang.String')
    housing_type_tag.text='House'
    
    root.append(household_tag)
    

    tree=ET.ElementTree(root)
    tree.write('SF_CHAMP_Converted/household_int_int_attributes.xml', pretty_print=True, xml_declaration=True,   encoding="utf-8")    

KeyboardInterrupt: 

In [32]:
df.head()

,hhno,person_id,hhvehs,hhincome,X_COORD,Y_COORD
0,1,1-1,0,942,551662.441744,4.173739e+06
1,10,10-1,0,28888,551662.441744,4.173739e+06
2,100,100-1,3,48612,551662.441744,4.173739e+06
3,100,100-2,3,48612,551662.441744,4.173739e+06
4,100,100-3,3,48612,551662.441744,4.173739e+06


In [21]:
%%time
df_w_veh=df[df['hhvehs']>0]

df_w_veh['veh_1']='auto'+'-'+df_w_veh['hhno']+'-'+'0'
df_w_veh['veh_2']=np.where(df_w_veh['hhvehs']>1,
                   'auto'+'-'+df_w_veh['hhno']+'-'+'1',
                  np.nan)
df_w_veh['veh_3']=np.where(df_w_veh['hhvehs']>2,
                   'auto'+'-'+df_w_veh['hhno']+'-'+'2',
                  np.nan)
df_w_veh['veh_4']=np.where(df_w_veh['hhvehs']>3,
                   'auto'+'-'+df_w_veh['hhno']+'-'+'3',
                  np.nan)
vehs=df_w_veh[['veh_1','veh_2','veh_3','veh_4']]
vehicle_1=vehs[['veh_1']]
vehicle_1=vehicle_1.rename(columns={'veh_1':'vehicleId'})
vehicle_2=vehs[vehs['veh_2'].notnull()][['veh_2']]
vehicle_2=vehicle_2.rename(columns={'veh_2':'vehicleId'})
vehicle_3=vehs[vehs['veh_3'].notnull()][['veh_3']]
vehicle_3=vehicle_3.rename(columns={'veh_3':'vehicleId'})
vehicle_4=vehs[vehs['veh_4'].notnull()][['veh_4']]
vehicle_4=vehicle_4.rename(columns={'veh_4':'vehicleId'})

df_1=pd.concat([vehicle_1,vehicle_2,vehicle_3,vehicle_4])
df_1['vehicleTypeId']='Car'
df_1.to_csv('../SF_CHAMP_Converted/vehicles.csv',index=False)

<timed exec>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
<timed exec>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
<timed exec>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
<timed exec>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a sli

Wall time: 5.96 s


In [172]:
def create_xml_pop_template(df):

    root = ET.Element('personsAttributes') 

    for i in df['person_id'].unique():
        
        new_df=df[df['person_id']==i]


        age_tag = ET.SubElement(person_tag, 'attribute')
        age_tag.set('name','age')
        age_tag.set('class','java.lang.Integer')
        age_tag.text=new_df.values[0,new_df.columns.get_loc('pagey')]
        #age_tag.set('age',df.values[0,df.columns.get_loc('pagey')])
        
        sex_tag = ET.SubElement(person_tag, 'attribute')
        sex_tag.set('name','sex')
        sex_tag.set('class','java.lang.String')
        sex_tag.text=new_df.values[0,new_df.columns.get_loc('gender')]
        #sex_tag.set('sex',df.values[0,df.columns.get_loc('gender')])
        
        modes_tag = ET.SubElement(person_tag, 'attribute')
        modes_tag.set('name','excluded-modes')
        modes_tag.set('class','java.lang.String')
        modes_tag.text=new_df.values[0,new_df.columns.get_loc('excluded_mode')]
        #modes_tag.text=np.where(new_df['excluded_mode'][0]=='0','',new_df['excluded_mode'][0])
#         if new_df['excluded_mode'][0]=='0':
#             modes_tag.text=''
#         else:
#             modes_tag.text=new_df['excluded_mode'][0]
        #modes_tag.set('excluded-modes',df.values[0,df.columns.get_loc('excluded_mode')])


        root.append(person_tag)


    tree=ET.ElementTree(root)
    tree.write('persons_attribute_int_int.xml', pretty_print=True, xml_declaration=True,   encoding="utf-8")

In [215]:
create_xml_pop_template(pop_df.head(1))

In [208]:
for i in range(1,21,5):
    
        print(i,i+5)

1 6
6 11
11 16
16 21


In [199]:
test=pop_df.iloc[5:10].reset_index()
test

,index,person_id,hhno,pagey,pgend,hhvehs,excluded_mode,gender
0,5,11-1,11,68,2,1,nan,female
1,6,10-1,10,51,1,0,car,male
2,7,3-1,3,47,1,0,car,male
3,8,31-1,31,60,1,1,nan,male
4,9,21-1,21,63,1,2,nan,male


In [205]:
test.values[4,test.columns.get_loc('pagey')]

'63'

In [211]:
def create_xml_pop(df_all,size):

    for i in range(1,len(df_all)+1,size):
    
        j=i+size

        test_df=df_all.iloc[i:j].reset_index()

        parser = ET.XMLParser(remove_blank_text=True)

        template = ET.parse('persons_attribute_int_int.xml',parser)

        root = template.getroot()
        
        for row in range(0,size-1):

            new_df=df[df['person_id']==i]


            age_tag = ET.SubElement(person_tag, 'attribute')
            age_tag.set('name','age')
            age_tag.set('class','java.lang.Integer')
            age_tag.text=new_df.values[0,new_df.columns.get_loc('pagey')]
            #age_tag.set('age',df.values[0,df.columns.get_loc('pagey')])

            sex_tag = ET.SubElement(person_tag, 'attribute')
            sex_tag.set('name','sex')
            sex_tag.set('class','java.lang.String')
            sex_tag.text=new_df.values[0,new_df.columns.get_loc('gender')]
            #sex_tag.set('sex',df.values[0,df.columns.get_loc('gender')])

            modes_tag = ET.SubElement(person_tag, 'attribute')
            modes_tag.set('name','excluded-modes')
            modes_tag.set('class','java.lang.String')
            modes_tag.text=new_df.values[0,new_df.columns.get_loc('excluded_mode')]
            #modes_tag.text=np.where(new_df['excluded_mode'][0]=='0','',new_df['excluded_mode'][0])
    #         if new_df['excluded_mode'][0]=='0':
    #             modes_tag.text=''
    #         else:
    #             modes_tag.text=new_df['excluded_mode'][0]
            #modes_tag.set('excluded-modes',df.values[0,df.columns.get_loc('excluded_mode')])


            root.append(person_tag)


        tree=ET.ElementTree(root)
        tree.write('persons_attribute_int_int.xml', pretty_print=True, xml_declaration=True,   encoding="utf-8")

In [216]:
%%time
create_xml_pop(pop_df,5000)

IndexError: index 11 is out of bounds for axis 0 with size 11

In [175]:
%%time
create_xml_pop_template(pop_df)

KeyboardInterrupt: 

In [ ]:
veh_list=[]
if df.hhvehs.max()>0:
    x=df.hhvehs.max()

    for item in range(0,x):
        vehicle_id='auto-'+hh_id+'-'+str(item)
        veh_list.append(vehicle_id)

    for vehicles in veh_list:
        vehicle_tag = ET.SubElement(household_tag, 'vehicleDefinitionID') 
        vehicle_tag.set('refId',str(vehicles))

In [ ]:
households_int_int=households[households['hhno'].isin(pop_df.hhno.unique())]

In [ ]:
df=household(households_int_int,person_int_int,mzone)

In [230]:
from lxml import etree as ET
def create_xml_test(hh_pop_mzone):

    root = ET.Element('households') 

    for i in hh_pop_mzone['hhno'].unique():
        
        hh_id=str(i)

        household_tag = ET.SubElement(root, 'household') 
        household_tag.set('id',str(hh_id))


        members_tag = ET.SubElement(household_tag, 'members')

        df=hh_pop_mzone[hh_pop_mzone['hhno']==i]
        
        person_list=df.person_id.unique()
        for person in df.person_id.unique():
            
            person_tag=ET.SubElement(members_tag,'personID')
            
            person_tag.set('refID',str(person))
        
        
        veh_list=[]
        if df.hhvehs.max()>0:
            x=df.hhvehs.max()
            
            for item in range(0,x):
                vehicle_id='auto-'+hh_id+'-'+str(item)
                veh_list.append(vehicle_id)
                
            for vehicles in veh_list:
                vehicle_tag = ET.SubElement(household_tag, 'vehicleDefinitionID') 
                vehicle_tag.set('refId',str(vehicles))
                
        income_tag=ET.SubElement(household_tag,'income')
        income_tag.set('period','year')
        income_tag.set('value',str(df.hhincome.max()))

        root.append(household_tag)


    tree=ET.ElementTree(root)
    tree.write('household_test.xml', pretty_print=True, xml_declaration=True,   encoding="utf-8")

In [231]:
create_xml_test(df_all.head(10))

In [235]:
from lxml import objectify

xml_data = objectify.parse('household_test.xml')  # Parse XML data
root = xml_data.getroot()  # Root element

data = []
cols=[]

for i in range(len(root.getchildren())):
    child = root.getchildren()[i]
    data.append([subchild.text for subchild in child.getchildren()])
    cols.append(child.tag)
    
df = pd.DataFrame(data)  # Create DataFrame and transpose it

In [252]:
root.findall('vehicleDefinitionID')

[]

In [255]:
df_w_veh=df_all[df_all['hhvehs']>0]
df_w_veh.head()

,hhno,person_id,hhvehs,hhincome,X_COORD,Y_COORD
3,5,5-1,1,18373,551662.441744,4.173739e+06
4,6,6-1,1,16311,551814.520039,4.173807e+06
6,8,8-1,1,31668,551662.441744,4.173739e+06
7,9,9-1,1,30020,551662.441744,4.173739e+06
9,11,11-1,1,36157,551662.441744,4.173739e+06


In [256]:
df_w_veh.dtypes

hhno          object
person_id     object
hhvehs         int64
hhincome       int64
X_COORD      float64
Y_COORD      float64
dtype: object

In [261]:
df_w_veh['veh_1']='auto'+'-'+df_w_veh['hhno']+'-'+'0'
df_w_veh['veh_2']=np.where(df_w_veh['hhvehs']>1,
                   'auto'+'-'+df_w_veh['hhno']+'-'+'1',
                  np.nan)
df_w_veh['veh_3']=np.where(df_w_veh['hhvehs']>2,
                   'auto'+'-'+df_w_veh['hhno']+'-'+'2',
                  np.nan)
df_w_veh['veh_4']=np.where(df_w_veh['hhvehs']>3,
                   'auto'+'-'+df_w_veh['hhno']+'-'+'3',
                  np.nan)

df_w_veh.head()

<ipython-input-261-a6874fb97696>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_w_veh['veh_1']='auto'+'-'+df_w_veh['hhno']+'-'+'0'
<ipython-input-261-a6874fb97696>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_w_veh['veh_2']=np.where(df_w_veh['hhvehs']>1,
<ipython-input-261-a6874fb97696>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.

,hhno,person_id,hhvehs,hhincome,X_COORD,Y_COORD,veh_1,veh_2,veh_3,veh_4
3,5,5-1,1,18373,551662.441744,4.173739e+06,auto-5-0,NaN,NaN,NaN
4,6,6-1,1,16311,551814.520039,4.173807e+06,auto-6-0,NaN,NaN,NaN
6,8,8-1,1,31668,551662.441744,4.173739e+06,auto-8-0,NaN,NaN,NaN
7,9,9-1,1,30020,551662.441744,4.173739e+06,auto-9-0,NaN,NaN,NaN
9,11,11-1,1,36157,551662.441744,4.173739e+06,auto-11-0,NaN,NaN,NaN


In [262]:
vehs=df_w_veh[['veh_1','veh_2','veh_3','veh_4']]
vehs.head()  

,veh_1,veh_2,veh_3,veh_4
3,auto-5-0,NaN,NaN,NaN
4,auto-6-0,NaN,NaN,NaN
6,auto-8-0,NaN,NaN,NaN
7,auto-9-0,NaN,NaN,NaN
9,auto-11-0,NaN,NaN,NaN


In [268]:
vehicle_1=vehs[['veh_1']]
vehicle_1=vehicle_1.rename(columns={'veh_1':'vehicle_id'})
vehicle_2=vehs[vehs['veh_2'].notnull()][['veh_2']]
vehicle_2=vehicle_2.rename(columns={'veh_2':'vehicle_id'})
vehicle_3=vehs[vehs['veh_3'].notnull()][['veh_3']]
vehicle_3=vehicle_3.rename(columns={'veh_3':'vehicle_id'})
vehicle_4=vehs[vehs['veh_4'].notnull()][['veh_4']]
vehicle_4=vehicle_4.rename(columns={'veh_4':'vehicle_id'})
vehicle_1.head()

,vehicle_id
3,auto-5-0
4,auto-6-0
6,auto-8-0
7,auto-9-0
9,auto-11-0


In [271]:
df_1=pd.concat([vehicle_1,vehicle_2,vehicle_3,vehicle_4])
df_1['vehicleTypeID']='Car'
df_1.to_csv('vehicles.csv')

In [59]:
trip=pd.read_table('_trip_2.dat',sep='\t',header='infer')
trip.head()

,id,tour_id,hhno,pno,day,tour,half,tseg,tsvid,opurp,dpurp,oadtyp,dadtyp,opcl,otaz,dpcl,dtaz,mode,pathtype,dorp,deptm,arrtm,endacttm,travtime,travcost,travdist,vot,trexpfac
0,9101,91,9,1,1,1,1,1,0,0,1,1,2,720,1,2673,440,3,2,1,419,435,934,16.59,0.5652,4.71,10.980627,1
1,9151,91,9,1,1,1,2,1,0,1,0,2,1,2673,440,720,1,3,2,1,934,950,179,16.67,0.5604,4.67,10.980627,1
2,5101,51,5,1,1,1,1,1,0,0,5,1,4,720,1,2239,924,6,3,4,578,610,632,32.07,0.4150,6.07,5.281956,1
3,5151,51,5,1,1,1,2,1,0,5,0,4,1,2239,924,720,1,6,3,4,632,663,179,31.57,0.4150,6.00,5.281956,1
4,18101,181,15,2,1,1,1,1,0,0,5,1,4,20,1,1301,918,3,2,1,902,917,926,15.52,0.5112,4.26,2.950609,1


In [60]:
trip_int_int=trip[(trip['otaz']<1000)&(trip['dtaz']<1000)]
trip_int_int=trip_int_int[['hhno']]
trip_int_int

,hhno
0,9
1,9
2,5
3,5
4,15
...,...
25042938,2837003
25042939,2837003
25042940,2837003
25044018,2834752


In [136]:
# trip['hhno']=trip['hhno'].astype(str)
# trip['pno']=trip['pno'].astype(str)

trip['person_id']=trip['hhno']+'-'+trip['pno']

trip_int_int_person=trip[(trip['otaz']<1000)&(trip['dtaz']<1000)]
trip_int_int_person=trip_int_int_person[['person_id']]
trip_int_int_person.head()


,person_id
0,9-1
1,9-1
2,5-1
3,5-1
4,15-2


In [61]:
households_int_int=households[households['hhno'].isin(trip_int_int.hhno.unique())]

In [138]:
# person['hhno']=person['hhno'].astype(str)
# person['pno']=person['pno'].astype(str)

person['person_id']=person['hhno']+'-'+person['pno']

person_int_int=person[person['person_id'].isin(trip_int_int_person.person_id.unique())]

In [139]:
person_int_int.head()

,id,hhno,pno,pptyp,pagey,pgend,pwtyp,pwpcl,pwtaz,pwautime,pwaudist,pstyp,pspcl,pstaz,psautime,psaudist,puwmode,puwarrp,puwdepp,ptpass,ppaidprk,pdiary,pproxy,psexpfac,person_id
5,9,9,1,1,54,1,1,2673,440,16.63,4.69,0,-1,-1,-1.0,-1.0,-1,-1,-1,0,1,-1,-1,1,9-1
6,5,5,1,3,79,2,0,-1,-1,-1.00,-1.00,0,-1,-1,-1.0,-1.0,-1,-1,-1,0,1,-1,-1,1,5-1
8,18,15,2,3,66,2,0,-1,-1,-1.00,-1.00,0,-1,-1,-1.0,-1.0,-1,-1,-1,1,1,-1,-1,1,15-2
10,28,20,2,4,55,1,0,-1,-1,-1.00,-1.00,0,-1,-1,-1.0,-1.0,-1,-1,-1,0,1,-1,-1,1,20-2
11,1,1,1,4,55,2,0,-1,-1,-1.00,-1.00,0,-1,-1,-1.0,-1.0,-1,-1,-1,0,1,-1,-1,1,1-1


In [62]:
households_int_int

,hhno,fraction_with_jobs_outside,hhsize,hhvehs,hhwkrs,hhftw,hhptw,hhret,hhoad,hhuni,hhhsc,hh515,hhcu5,hhincome,hownrent,hrestype,hhparcel,zone_id,hhtaz,hhexpfac,samptype
3,9,0.04796,1,1,1,1,0,0,0,0,0,0,0,30020,11117,3,720,0,1,1,11
4,5,0.04796,1,1,0,0,0,1,0,0,0,0,0,18373,11117,2,720,0,1,1,11
5,15,0.04796,2,1,0,0,0,2,0,0,0,0,0,9165,11488,3,20,0,1,1,11
6,20,0.04796,2,1,1,1,0,0,1,0,0,0,0,36487,11117,1,720,0,1,1,11
7,1,0.04796,1,0,0,0,0,0,1,0,0,0,0,942,11117,2,720,0,1,1,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2836223,2814016,0.00818,1,1,1,0,0,0,0,1,0,0,0,13578,30305,5,314215,1802,2012,1,13
2836614,2834891,0.00907,1,1,1,0,1,0,0,0,0,0,0,7662,273,5,813205,2208,2418,1,13
2836777,2837003,0.00000,1,1,0,0,0,0,1,0,0,0,0,23457,76,5,813000,2243,2453,1,13
2837159,2834752,0.00340,1,1,1,0,1,0,0,0,0,0,0,7859,9451,5,813523,2202,2412,1,13


In [63]:
df=household(households_int_int,person,mzone)
df.head()

,hhno,person_id,hhvehs,hhincome,X_COORD,Y_COORD
0,1,1-1,0,942,551662.441744,4.173739e+06
1,3,3-1,0,8728,551814.520039,4.173807e+06
2,4,4-1,0,5052,551662.441744,4.173739e+06
3,5,5-1,1,18373,551662.441744,4.173739e+06
4,6,6-1,1,16311,551814.520039,4.173807e+06


In [11]:
df_1=df.head(1)

In [71]:
df_all=df[df['hhno']!='1']

In [72]:
df_all

,hhno,person_id,hhvehs,hhincome,X_COORD,Y_COORD
1,3,3-1,0,8728,551814.520039,4.173807e+06
2,4,4-1,0,5052,551662.441744,4.173739e+06
3,5,5-1,1,18373,551662.441744,4.173739e+06
4,6,6-1,1,16311,551814.520039,4.173807e+06
5,7,7-1,0,11451,551662.441744,4.173739e+06
...,...,...,...,...,...,...
1176194,2837003,2837003-1,1,23457,538448.122055,4.190256e+06
1176195,2837007,2837007-1,0,0,541825.205926,4.187812e+06
1176196,2837035,2837035-1,1,3649,527226.292042,4.195602e+06
1176197,2837166,2837166-1,1,0,527226.292042,4.195602e+06


In [14]:
create_xml(df_1)

In [26]:
%%time
from lxml import etree as ET


for i in range(0,len(df_all)+5000,5000):
    
    j=i-5000
    
    test_df=df_all[df_all['hhno'].isin(df_all.hhno.unique()[j:i])]

    parser = ET.XMLParser(remove_blank_text=True)

    template = ET.parse('household_int_int.xml',parser)
    
    root = template.getroot() 

    for i in hh_pop_mzone['hhno'].unique():
        
        hh_id=str(i)

        household_tag = ET.SubElement(root, 'household') 
        household_tag.set('id',str(hh_id))


        members_tag = ET.SubElement(household_tag, 'members')

        df=hh_pop_mzone[hh_pop_mzone['hhno']==i]
        
        person_list=df.person_id.unique()
        for person in df.person_id.unique():
            
            person_tag=ET.SubElement(members_tag,'personId')
            
            person_tag.set('refId',str(person))
        
        
        
        veh_list=[]
        if df.hhvehs.max()>0:
            x=df.hhvehs.max()
            vehicle_tag=ET.SubElement(household_tag, 'vehicles')
            
            for item in range(0,x):
                vehicle_id='auto-'+hh_id+'-'+str(item)
                veh_list.append(vehicle_id)
                
            for vehicles in veh_list:
                
                vehicle_list_tag = ET.SubElement(vehicle_tag, 'vehicleDefinitionId')
                vehicle_list_tag.set('refId',str(vehicles))
                
        income_tag=ET.SubElement(household_tag,'income')
        income_tag.set('period','year')
        income_tag.set('currency','usd')
        income_tag.text=str(df.hhincome.max())
        #income_tag.set('value',str(df.hhincome.max()))

        root.append(household_tag)


    tree=ET.ElementTree(root)
    tree.write('SmallSample/household_int_int.xml', pretty_print=True, xml_declaration=True,   encoding="utf-8")




Wall time: 40min 33s


In [13]:
from lxml import etree as ET
def create_xml(hh_pop_mzone):

    root = ET.Element('households') 

    for i in hh_pop_mzone['hhno'].unique():
        
        hh_id=str(i)

        household_tag = ET.SubElement(root, 'household') 
        household_tag.set('id',str(hh_id))


        members_tag = ET.SubElement(household_tag, 'members')

        df=hh_pop_mzone[hh_pop_mzone['hhno']==i]
        
        person_list=df.person_id.unique()
        for person in df.person_id.unique():
            
            person_tag=ET.SubElement(members_tag,'personID')
            
            person_tag.set('refID',str(person))
        
        
        veh_list=[]
        if df.hhvehs.max()>0:
            x=df.hhvehs.max()
            
            for item in range(0,x):
                vehicle_id='auto-'+hh_id+'-'+str(item)
                veh_list.append(vehicle_id)
                
            for vehicles in veh_list:
                vehicle_tag = ET.SubElement(household_tag, 'vehicleDefinitionID') 
                vehicle_tag.set('refId',str(vehicles))
                
        income_tag=ET.SubElement(household_tag,'income')
        income_tag.set('period','year')
        income_tag.set('value',str(df.hhincome.max()))

        root.append(household_tag)


    tree=ET.ElementTree(root)
    tree.write('household_int_int.xml', pretty_print=True, xml_declaration=True,   encoding="utf-8")

In [10]:
create_xml(df)

KeyboardInterrupt: 

In [118]:
df_test=df_all.reset_index()
df_test

,index,hhno,person_id,hhvehs,hhincome,X_COORD,Y_COORD
0,1,3,3-1,0,8728,551814.520039,4.173807e+06
1,2,4,4-1,0,5052,551662.441744,4.173739e+06
2,3,5,5-1,1,18373,551662.441744,4.173739e+06
3,4,6,6-1,1,16311,551814.520039,4.173807e+06
4,5,7,7-1,0,11451,551662.441744,4.173739e+06
...,...,...,...,...,...,...,...
1176193,1176194,2837003,2837003-1,1,23457,538448.122055,4.190256e+06
1176194,1176195,2837007,2837007-1,0,0,541825.205926,4.187812e+06
1176195,1176196,2837035,2837035-1,1,3649,527226.292042,4.195602e+06
1176196,1176197,2837166,2837166-1,1,0,527226.292042,4.195602e+06


In [127]:
df_test.hhno.unique()[0:10]

array(['3', '4', '5', '6', '7', '8', '9', '10', '11', '12'], dtype=object)

In [126]:
df_all.hhno.unique()

array(['3', '4', '5', ..., '2837035', '2837166', '2837191'], dtype=object)

In [130]:
%%time
from lxml import etree as ET


for i in range(0,len(df_all.hhno.unique())+5000,5000):
    
    j=i+5000
    
    test_df=df_all[df_all['hhno'].isin(df_all.hhno.unique()[i:j])]

    parser = ET.XMLParser(remove_blank_text=True)

    template = ET.parse('household_int_int_attributes.xml',parser)
    root = template.getroot()

    for hh in test_df['hhno'].unique():
        
        hh_id=str(hh)
    
        household_tag = ET.SubElement(root, 'household') 
        household_tag.set('id',str(hh_id))

        subset_df=df_all[df_all['hhno']==hh]

        xcoord_tag = ET.SubElement(household_tag, 'homexcoord')
        xcoord_tag.set('x',str(subset_df['X_COORD'].unique()[0]))

        ycoord_tag = ET.SubElement(household_tag, 'homeycoord')
        ycoord_tag.set('y',str(subset_df['Y_COORD'].unique()[0]))

        root.append(household_tag)

        


    tree=ET.ElementTree(root)
    tree.write('household_int_int_attributes.xml', pretty_print=True, xml_declaration=True,   encoding="utf-8")


Wall time: 12h 3min 47s


In [116]:
%%time
root=ET.Element('householdAttributes')

for i in df_all['hhno'].unique():
    
    hh_id=str(i)
    
    household_tag = ET.SubElement(root, 'household') 
    household_tag.set('id',str(hh_id))

    subset_df=df_all[df_all['hhno']==i]
    
    xcoord_tag = ET.SubElement(household_tag, 'homexcoord')
    xcoord_tag.set('x',str(subset_df['X_COORD'].unique()[0]))
    
    ycoord_tag = ET.SubElement(household_tag, 'homeycoord')
    ycoord_tag.set('y',str(subset_df['Y_COORD'].unique()[0]))
    
    root.append(household_tag)
    

tree=ET.ElementTree(root)
tree.write('household_int_int_attributes.xml', pretty_print=True, xml_declaration=True,   encoding="utf-8")    

KeyboardInterrupt: 

In [42]:

df_all.drop(columns=['level_0'],inplace=True)
df_all.head()

,hhno,person_id,hhvehs,hhincome,X_COORD,Y_COORD
0,3,3-1,0,8728,551814.520039,4.173807e+06
1,4,4-1,0,5052,551662.441744,4.173739e+06
2,5,5-1,1,18373,551662.441744,4.173739e+06
3,6,6-1,1,16311,551814.520039,4.173807e+06
4,7,7-1,0,11451,551662.441744,4.173739e+06


In [101]:
df_all.to_csv('households.csv')

In [113]:
%%time
from lxml import etree as ET

for i in range(0,len(df_all)+5000,5000):
    
    j=i+5000
    
    test_df=df_all[df_all['hhno'].isin(df_all.hhno.unique()[i:j])]

    parser = ET.XMLParser(remove_blank_text=True)

    template = ET.parse('household_int_int_attributes.xml',parser)
    
    root = template.getroot() 

    for hh_unique in test_df['hhno'].unique():
        
        hh_id=str(i)
    
        household_tag = ET.SubElement(root, 'household') 
        household_tag.set('id',str(hh_id))

        subset_df=test_df[test_df['hhno']==i]

        xcoord_tag = ET.SubElement(household_tag, 'homexcoord')
        xcoord_tag.set('x',str(subset_df['X_COORD'].unique()[0]))

        ycoord_tag = ET.SubElement(household_tag, 'homeycoord')
        ycoord_tag.set('y',str(subset_df['Y_COORD'].unique()[0]))

        root.append(household_tag)


    tree=ET.ElementTree(root)
    tree.write('household_int_int_attributes.xml', pretty_print=True, xml_declaration=True,   encoding="utf-8")





IndexError: index 0 is out of bounds for axis 0 with size 0